In [16]:

######## prompting ONLY ########


import base64
from pathlib import Path
from openai import OpenAI
from tqdm import tqdm

API_KEY = "" #api key 放在這

client = OpenAI(api_key=API_KEY)


output_file = Path("/.../GPT_output_prompt_only.txt") #先開一個GPT_output_prompt_only.txt的txt檔案放在桌面
output_file.parent.mkdir(parents=True, exist_ok=True) 

SEPARATOR = "\n\n\n——\n\n\n"

prompt = '''I have a dataset in an arbitrary language, but because its script is particularly difficult to format for computer input, different <token_N>, where N is the number, will represent different characters. For example, |<token_1><token_3><token_6>| can substitute for a string or sentence composed of three characters. Please use this as background knowledge for handling the following questions.

The Avoiuli writing system is used on Pentecost Island in Vanuatu, a Pacific Island Country in the Pacific region of Melanesia. With a design inspired by traditional sand drawings, Avoiuli was developed over fourteen years by Chief Viraleo Boborenvanua, as part of a movement he leads to revitalize Melanesian culture and language.
One of the languages that Avoiuli is used to write is Bislama, one of the official languages of Vanuatu. As you will see, Bislama is a creole language – many of its words come from English, some come from French, and others from the indigenous languages of Vanuatu.
Below is a sign written in Bislama, using the Avoiuli script, on the front of a college on Pentecost Island. For the purposes of this problem, each of the words in the script below has been allotted a number:

1. <token_1><token_2><token_3>
2. <token_1><token_4><token_5><token_6><token_2>
3. <token_3><token_7><token_8><token_9>
4. <token_6><token_10><token_11><token_6><token_11><token_6><token_1><token_5><token_11>
5. <token_5><token_2><token_11><token_1><token_11><token_5><token_12><token_13><token_12><token_14>
6. <token_3><token_7><token_8><token_9>
7. <token_6><token_15><token_16><token_15><token_14>
8. <token_1><token_2><token_4><token_5>
9. <token_17><token_15><token_7><token_8><token_1><token_8><token_17><token_15>
10. <token_11><token_16><token_8><token_13><token_8><token_5><token_18><token_12><token_6>
11. <token_8><token_14>
12. <token_11><token_6><token_11><token_5><token_2><token_14><token_10><token_11><token_19>
13. <token_7><token_15><token_1><token_2>
14. <token_20><token_15><token_7><token_15><token_16>
15. <token_7><token_8><token_7><token_8><token_20><token_15><token_5><token_15>

Q 6.1 On your answer sheet, give the number of a word (any one, if there are multiple) which corresponds to these Bislama words:

(a) filosofi 

(b) institiut

(c) blong

Q 6.2 Give the numbers of a word (any one, if there are multiple) which means:

(a) village 

(b) centre 

(c) teach

(d) technology

(e) humanity 

Q 6.3 Which geographical area of the Pacific is mentioned in the text?'''

image_path = None  # 沒圖片就設成 None


def image_to_data_url(path: Path) -> str:
    b64 = base64.b64encode(path.read_bytes()).decode("utf-8")
    ext = path.suffix.lstrip(".").lower() or "png"
    return f"data:image/{ext};base64,{b64}"

for i in tqdm(range(2), desc="Calling OpenAI API"): #range(數字) 代表你要跑幾次 自己改
    content = [{"type": "text", "text": prompt}]

    if image_path is not None:
        content.append({
            "type": "image_url",
            "image_url": {
                "url": image_to_data_url(image_path)
            }
        })

    response = client.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "user", "content": content}
        ],
    )

    reply = response.choices[0].message.content or ""
    print(reply)
    # append 到檔案
    if output_file.stat().st_size > 0:
        reply = SEPARATOR + reply

    with output_file.open("a", encoding="utf-8") as f:
        f.write(f"[run {i}]\n{reply}")



Calling OpenAI API:  50%|████████████▌            | 1/2 [00:07<00:07,  7.61s/it]

### Q 6.1  (give a word number)

To answer these, we match repeated Avoiuli “words” to repeated Bislama words likely to occur on a college sign.

- **(c) _blong_** is very common in Bislama signage (means “of / belonging to”), and it’s likely a short standalone word. The best match is **word 11**:  
  **11.** `<token_8><token_14>`

For the other two, we look for longer “loanword-shaped” items:

- **(a) _filosofi_** looks like “philosophy” and is fairly long; the best candidate among the long words is **word 12**.  
  **(a) filosofi → 12**

- **(b) _institiut_** (“institute”) is also long; the best candidate is **word 10**.  
  **(b) institiut → 10**

So:

- (a) **12**  
- (b) **10**  
- (c) **11**

---

### Q 6.2  (give word numbers)

Using common Bislama vocabulary expected on an educational sign (e.g., “village”, “centre”, “teach”, “technology”, “humanity”) and matching by plausible word length/pattern, the best matches are:

- **(a) village → 13**  
- **(b) centre → 8**  
- **(c) te

Calling OpenAI API: 100%|█████████████████████████| 2/2 [00:11<00:00,  5.50s/it]

### Q 6.1  (give the number of a word)

Because Bislama words on institutional signs are often recognizable borrowings from English/French, we can match by “shape” (length/pattern) and repetition:

- **(a) filosofi** → **word 15**  
- **(b) institiut** → **word 12**  
- **(c) blong** → **word 2**

---

### Q 6.2  (give the number of a word meaning …)

- **(a) village** → **word 13**
- **(b) centre** → **word 14**
- **(c) teach** → **word 4**
- **(d) technology** → **word 10**
- **(e) humanity** → **word 5**

---

### Q 6.3  Geographical area of the Pacific mentioned

**Melanesia**.


In [11]:

######## with 圖片 ########

import base64
from pathlib import Path
from openai import OpenAI
from tqdm import tqdm

API_KEY = "" #api key 放在這 

client = OpenAI(api_key=API_KEY)


output_file = Path("/Users/linzhenglin/Desktop/GPT_output_with_pic.txt") #先開一個GPT_output_with_pic.txt的txt檔案放在桌面 記得把檔案位置改成你的檔案位置
output_file.parent.mkdir(parents=True, exist_ok=True)

SEPARATOR = "\n\n\n——\n\n\n"

prompt = '''I have a dataset in an arbitrary language, but because its script is particularly difficult to format for computer input, different <token_N>, where N is the number, will represent different characters. For example, |<token_1><token_3><token_6>| can substitute for a string or sentence composed of three characters. Please use this as background knowledge for handling the following questions. 
The appearance of each token is provided in the given image.

The Avoiuli writing system is used on Pentecost Island in Vanuatu, a Pacific Island Country in the Pacific region of Melanesia. With a design inspired by traditional sand drawings, Avoiuli was developed over fourteen years by Chief Viraleo Boborenvanua, as part of a movement he leads to revitalize Melanesian culture and language.
One of the languages that Avoiuli is used to write is Bislama, one of the official languages of Vanuatu. As you will see, Bislama is a creole language – many of its words come from English, some come from French, and others from the indigenous languages of Vanuatu.
Below is a sign written in Bislama, using the Avoiuli script, on the front of a college on Pentecost Island. For the purposes of this problem, each of the words in the script below has been allotted a number:

1. <token_1><token_2><token_3>
2. <token_1><token_4><token_5><token_6><token_2>
3. <token_3><token_7><token_8><token_9>
4. <token_6><token_10><token_11><token_6><token_11><token_6><token_1><token_5><token_11>
5. <token_5><token_2><token_11><token_1><token_11><token_5><token_12><token_13><token_12><token_14>
6. <token_3><token_7><token_8><token_9>
7. <token_6><token_15><token_16><token_15><token_14>
8. <token_1><token_2><token_4><token_5>
9. <token_17><token_15><token_7><token_8><token_1><token_8><token_17><token_15>
10. <token_11><token_16><token_8><token_13><token_8><token_5><token_18><token_12><token_6>
11. <token_8><token_14>
12. <token_11><token_6><token_11><token_5><token_2><token_14><token_10><token_11><token_19>
13. <token_7><token_15><token_1><token_2>
14. <token_20><token_15><token_7><token_15><token_16>
15. <token_7><token_8><token_7><token_8><token_20><token_15><token_5><token_15>

Q 6.1 On your answer sheet, give the number of a word (any one, if there are multiple) which corresponds to these Bislama words:

(a) filosofi 

(b) institiut

(c) blong

Q 6.2 Give the numbers of a word (any one, if there are multiple) which means:

(a) village 

(b) centre 

(c) teach

(d) technology

(e) humanity 

Q 6.3 Which geographical area of the Pacific is mentioned in the text?'''

image_path = Path("/Users/linzhenglin/Desktop/avo.png")  # 這裡放你的token圖片位置


def image_to_data_url(path: Path) -> str:
    b64 = base64.b64encode(path.read_bytes()).decode("utf-8")
    ext = path.suffix.lstrip(".").lower() or "png"
    return f"data:image/{ext};base64,{b64}"

for i in tqdm(range(5), desc="Calling OpenAI API"): #range(數字) 代表你要跑幾次 自己改
    content = [{"type": "text", "text": prompt}]

    if image_path is not None:
        content.append({
            "type": "image_url",
            "image_url": {
                "url": image_to_data_url(image_path)
            }
        })

    response = client.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "user", "content": content}
        ],
    )

    reply = response.choices[0].message.content or ""
    print(reply)
    # append 到檔案
    if output_file.stat().st_size > 0:
        reply = SEPARATOR + reply

    with output_file.open("a", encoding="utf-8") as f:
        f.write(f"[run {i}]\n{reply}")



### Q6.1  (give the word number)

To match these, use the fact that many Bislama words are close to English/French loanwords, so their Avoiuli spellings repeat in the sign:

- **(a) filosofi** → **15**  
- **(b) institiut** → **4**  
- **(c) blong** → **11**

---

### Q6.2  (give the word number)

- **(a) village** → **1**  
- **(b) centre** → **8**  
- **(c) teach** → **13**  
- **(d) technology** → **10**  
- **(e) humanity** → **12**

---

### Q6.3

The geographical area mentioned is **Melanesia**.
Done.
